In [21]:
import keras
import numpy as np
import tensorflow as tf

In [22]:
from sklearn.metrics  import *
from sklearn.datasets import load_breast_cancer

In [23]:
X, y = load_breast_cancer(return_X_y=True)

In [24]:
X = (X - X.mean(axis=0)) / X.std(axis=0)

In [25]:
X = X.astype(np.float32)
y = y.astype(np.float32)

In [26]:
X.shape[0]//32, X.shape[1]

(17, 30)

In [27]:
data = tf.data.Dataset.from_tensor_slices((X, y.reshape(-1, 1)))

In [28]:
train = data\
            .batch( batch_size = 20, drop_remainder=True)\
            .shuffle(buffer_size = 20, seed=21)

## Classificação

In [29]:
class modelClass_(tf.Module):
    def __init__(self, name=None, initializer_ = keras.initializers.GlorotNormal, activation=tf.sigmoid,n_features=8):
        super().__init__(name)

        self.activation = activation
        initializer_ = initializer_()
        initializer_ = initializer_(shape=(n_features, 1))

        # Pesos e bias
        self.weights = tf.Variable(initializer_, name='weights')
        self.bias    = tf.Variable([0.0], name='bias')

    def __call__(self, x):
        return self.activation(tf.matmul(x, self.weights) + self.bias)

In [30]:
model = modelClass_(
    name='meu_modelo', 
    n_features=30    ,
    initializer_=keras.initializers.Zeros)

In [31]:
X.shape

(569, 30)

In [32]:
loss_  = keras.losses.binary_crossentropy
metric = keras.metrics.binary_accuracy
optim  = keras.optimizers.SGD(learning_rate=0.01)

max_ = np.inf
init_, limit = 0, 4

for epoch in range(10):
    for xbatch, ybatch in train:
        with tf.GradientTape() as tape:
            pred = model(xbatch)
            loss = loss_(ybatch, pred)
            loss = tf.reduce_mean(loss)

              
        gradient = tape.gradient(loss, model.variables)
        optim.apply_gradients(zip(gradient, model.variables))
    if max_ > loss:
        init_ += 1
    if init_ == limit + 1:
        break
    print(f'Epoch {epoch+1} * loss : {loss:.2f}')
    

        

Epoch 1 * loss : 0.45
Epoch 2 * loss : 0.34
Epoch 3 * loss : 0.24
Epoch 4 * loss : 0.12


In [33]:
pred_ = (model(X) > 0.5).numpy().astype(float)

In [34]:
accuracy_score(y, pred_)

0.9595782073813708

In [35]:
print(classification_report(y, pred_))

              precision    recall  f1-score   support

         0.0       0.94      0.95      0.95       212
         1.0       0.97      0.97      0.97       357

    accuracy                           0.96       569
   macro avg       0.96      0.96      0.96       569
weighted avg       0.96      0.96      0.96       569



## Regressão

In [36]:
from sklearn.datasets import fetch_california_housing

In [37]:
X, y = fetch_california_housing(return_X_y=True)

In [ ]:
X = (X - X.mean(axis=0)) / X.std(axis=0)

X = X.astype(np.float32)
y = y.astype(np.float32)

In [40]:
X.shape[0]//10, X.shape[1]

(2064, 8)

In [ ]:
data = tf.data\
         .Dataset.from_tensor_slices((X, y.reshape(-1, 1)))

train = data\
            .batch( batch_size = 32, drop_remainder=True)\
            .shuffle(buffer_size = 20, seed=21)

In [43]:
class modelReg_(tf.Module):
    def __init__(self, name=None, initializer_ = keras.initializers.VarianceScaling, dist='normal',n_features=8):
        super().__init__(name)

        if isinstance(initializer_, keras.initializers.VarianceScaling):
            initializer_ = initializer_(scale=1., mode='fan_in', distribution='normal')
            initializer_ = initializer_(shape=(n_features, 1))
        else:
            initializer_ = initializer_()
            initializer_ = initializer_(shape=(n_features, 1))

        # Pesos e bias
        self.weights = tf.Variable(initializer_, name='weights')
        self.bias    = tf.Variable([0.0], name='bias')

    def __call__(self, x):
        return tf.matmul(x, self.weights) + self.bias

In [44]:
model_reg = modelReg_(
    name = 'Regressao',
    n_features = 8, 
    initializer_ = keras.initializers.RandomNormal)

In [ ]:
loss_  = keras.losses.mean_squared_error
optim  = keras.optimizers.Nadam(clipnorm = 2)

max_ = np.inf
init_, limit = 0, 4

for epoch in range(10):
    for xbatch, ybatch in train:
        with tf.GradientTape() as tape:
            pred = model_reg(xbatch)
            loss = loss_(ybatch, pred)
            loss = tf.reduce_mean(loss)
                
        gradient = tape.gradient(loss, model_reg.variables)
        optim.apply_gradients(zip(gradient, model_reg.variables))
    if max_ > loss:
        init_ += 1
    if init_ == limit + 1:
        break
    print(f'Epoch {epoch+1} * loss : {loss:.2f}')

Epoch 1 * loss : 5.82
